###### Collect Dataset

In [14]:
import cv2
import numpy as np
import os
import dlib
from imutils import face_utils
from imutils.face_utils import FaceAligner

detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
face_aligner = FaceAligner(shape_predictor, desiredFaceWidth=200)

#face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

video_capture = cv2.VideoCapture(0)

name = input("Enter name of person:")

path = 'Dataset'
print(path)
directory = os.path.join(path, name)
print(directory)
if not os.path.exists(directory):
    os.makedirs(directory, exist_ok = 'True')

number_of_images = 0
MAX_NUMBER_OF_IMAGES = 100
count = 0

while number_of_images < MAX_NUMBER_OF_IMAGES:
    ret, frame = video_capture.read()

    frame = cv2.flip(frame, 1)

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    faces = detector(frame_gray)
    if len(faces) == 1:
        face = faces[0]
        (x, y, w, h) = face_utils.rect_to_bb(face)
        face_img = frame_gray[y-50:y + h+100, x-50:x + w+100]
        # Calculate the angle between the eyes
        dY = right_eye.y - left_eye.y
        dX = right_eye.x - left_eye.x
        angle = np.degrees(np.arctan2(dY, dX)) - 180

# Calculate the scale for resizing
        desiredFaceWidth = 200
        scale = desiredFaceWidth / w

# Calculate the rotation matrix
        M = cv2.getRotationMatrix2D(eyes_center, angle, scale)

# Apply the rotation to the cropped face region
        face_aligned = cv2.warpAffine(face_region_gray, M, (desiredFaceWidth, h))

        if count == 5:
            cv2.imwrite(os.path.join(directory, str(name+str(number_of_images)+'.jpg')), face_aligned)
            number_of_images += 1
            count = 0
        count += 1

#         face_aligned = face_aligner.align(frame, frame_gray, face)

#         if count == 5:
#             cv2.imwrite(os.path.join(directory, str(name+str(number_of_images)+'.jpg')), face_aligned)
#             number_of_images += 1
#             count = 0
#         #print(count)
#         count+=1
        
        
    cv2.imshow('Collecting the Face Data', frame)
    

    if(cv2.waitKey(1) & 0xFF == ord('q')):
        break

video_capture.release()
cv2.destroyAllWindows()
print("Dataset completed!...")


Enter name of person: qwer


Dataset
Dataset\qwer


NameError: name 'face_region_gray' is not defined

###### Train The Model

In [ ]:
import os
import math
from sklearn import neighbors
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder
import pickle
def train(train_dir, model_save_path, n_neighbors=2, knn_algo='ball_tree', verbose=False):
    X = []
    y = []
    # Loop through each person in the training set
    for class_dir in os.listdir(train_dir):
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            continue
        # Loop through each training image for the current person
        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
            image = face_recognition.load_image_file(img_path)
            face_bounding_boxes = face_recognition.face_locations(image)
            #print( "processing :",img_path)
            if len(face_bounding_boxes) != 1:
                # If there are no people (or too many people) in a training image, skip the image.
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                # Add face encoding for current image to the training set
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)

    # Determine how many neighbors to use for weighting in the KNN classifier
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)
    # Create and train the KNN classifier
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)
    # Save the trained KNN classifier
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)
        print("Training completed!...")
    return knn_clf
train("Dataset","classifier/trained_knn_model.clf") # add path here


###### Face Recognition

In [ ]:
import cv2,time
import pickle
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder
def predict(img_path, knn_clf=None, model_path=None, threshold=0.6): # 6 needs 40+ accuracy, 4 needs 60+ accuracy
    if knn_clf is None and model_path is None:
        raise Exception("Must supply knn classifier either thourgh knn_clf or model_path")
    # Load a trained KNN model (if one was passed in)
    if knn_clf is None:
        with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)
    # Load image file and find face locations
    img = img_path
    face_box = face_recognition.face_locations(img)
    # If no faces are found in the image, return an empty result.
    if len(face_box) == 0:
        return []
    # Find encodings for faces in the test iamge
    faces_encodings = face_recognition.face_encodings(img, known_face_locations=face_box)
    # Use the KNN model to find the best matches for the test face
    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=2)
    matches = [closest_distances[0][i][0] <= threshold for i in range(len(face_box))]
    accu=[100-closest_distances[0][i][0]*100 for i in range(len(face_box))]
    # Predict classes and remove classifications that aren't within the threshold
    return [(pred, loc,accu) if rec else ("Unknown Face ", loc,accu) for pred, loc, rec,accu in zip(knn_clf.predict(faces_encodings),face_box,matches,accu)]
webcam = cv2.VideoCapture(0) #  0 to use webcam 
while True:
    # Loop until the camera is working
    rval = False
    while(not rval):
        # Put the image from the webcam into 'frame'
        (rval, frame) = webcam.read()
        if(not rval):
            print("Failed to open webcam. Trying again...")


    # start_frame=time.time()        
    # Flip the image (optional)
    frame=cv2.flip(frame,1) # 0 = horizontal ,1 = vertical , -1 = both
    frame_copy = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    frame_copy=cv2.cvtColor(frame_copy, cv2.COLOR_BGR2RGB)
    predictions = predict(frame_copy, model_path="classifier/trained_knn_model.clf")
    font = cv2.FONT_HERSHEY_DUPLEX
    for name, (top, right, bottom, left),accuracy in predictions:
        top *= 4 #scale back the frame since it was scaled to 1/4 in size
        right *= 4
        bottom *= 4
        left *= 4
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        # name=("{}{:.2f}".format(name,accuracy))
        name = "{} {:.2f}".format(name, accuracy)
        # size = (right-left)/220
        cv2.putText(frame, name, (left-10,top-6), font, 0.8, (255, 255, 255), 1)
        #print(name)
        # end_time=time.time()
        # fps = 1/(end_time-start_frame)
        # cv2.rectangle(frame,(15,20),(125,45),(0,255,255),-1)
        # fps ="fps:{:.2f}".format(fps)

        # cv2.putText(frame,fps,(16,40),font,0.7,(0,0,0),1)

    cv2.imshow('Face Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
webcam.release()
cv2.destroyAllWindows()